In [ ]:
"""This is a classic implementation of Strategy pattern of design.
Using classes and methods. It is summarized in the design pattern
book as:
Define a family of algorithms, encapsulate each one, and make them
interchangeable. Strategy lets the algorithm vary independently from 
clients that use it."""

In [ ]:
"""
Context: provides a service by delegating some computation to 
interchangeable components.

Strategy: The common interface to the components.

Concrete Strategy: One of the subclasses of Strategy.
"""

In [8]:
from abc import  ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity


class Order:  # The Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# In python 3.4 simplest way to define it as abstract base class
# class name(ABC)
# Python <= 3.3 class name(metaclass=ABCMeta)


class Promotion(ABC):  # the strategy: an Abstract Base Class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive currency amount"""


class FidelityPromo(Promotion):  # First concrete strategy
    """5% discount for customers with 1000 or more fidelity points"""
    
    def discount(self, order):
        return order.total() * 0.05 if \
            order.customer.fidelity >= 1000 else 0
   
    
class BulkItemPromo(Promotion):  # Second concrete strategy
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        return sum(item.total() * 0.1 for item in order.cart 
                   if item.quantity >= 20)


class LargeOrderPromo(Promotion):  # Third concrete strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [3]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [4]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [5]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, 0.05),
               LineItem('apple', 10, 1.5)]

In [9]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 16.50 due: 16.35>

In [10]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [11]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>